In [1]:
!pip install  -q git+https://github.com/huggingface/transformers

In [2]:
!pip install peft -q

In [3]:
!pip install -q datasets

In [4]:
!pip install accelerate bitsandbytes -q

In [5]:

from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers.models.phi import PhiForCausalLM
import torch

In [6]:
tokenizer = AutoTokenizer.from_pretrained(
    "susnato/phi-2",
    trust_remote_code = True
)

tokenizer_config.json:   0%|          | 0.00/7.34k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [7]:
model = PhiForCausalLM.from_pretrained(
    "susnato/phi-2",
    torch_dtype = torch.float32 ,
    trust_remote_code = True,
)
     

The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.


config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/5.56G [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


generation_config.json:   0%|          | 0.00/69.0 [00:00<?, ?B/s]

In [24]:
model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): PhiForCausalLM(
      (model): PhiModel(
        (embed_tokens): Embedding(51200, 2560)
        (embed_dropout): Dropout(p=0.0, inplace=False)
        (layers): ModuleList(
          (0-47): 48 x PhiDecoderLayer(
            (self_attn): PhiAttention(
              (query_key_value): lora.Linear(
                (base_layer): Linear(in_features=2560, out_features=7680, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2560, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=7680, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (dense): Linear(in_feat

In [9]:
import copy


In [10]:
new_model = copy.deepcopy(model)

In [11]:
num_layers_per_model = new_model.model.config.num_hidden_layers - 8

In [12]:
num_layers_per_model

24

In [13]:
layers =new_model.model.layers
len(layers)

32

In [14]:
new_layers = new_model.model.layers[:num_layers_per_model] + model.model.layers[8:]

In [15]:
new_layers

ModuleList(
  (0-47): 48 x PhiDecoderLayer(
    (self_attn): PhiAttention(
      (query_key_value): Linear(in_features=2560, out_features=7680, bias=True)
      (dense): Linear(in_features=2560, out_features=2560, bias=True)
      (attention_dropout): Dropout(p=0.0, inplace=False)
      (rotary_emb): PhiRotaryEmbedding()
    )
    (mlp): PhiMLP(
      (activation_fn): NewGELUActivation()
      (fc1): Linear(in_features=2560, out_features=10240, bias=True)
      (fc2): Linear(in_features=10240, out_features=2560, bias=True)
    )
    (input_layernorm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
    (resid_dropout): Dropout(p=0.1, inplace=False)
  )
)

In [16]:
for i, n in enumerate(new_layers):
    n.self_attn.layer_idx = i

In [17]:
new_model.model.layers = new_layers

In [18]:
new_model = new_model.to('cuda')

In [19]:
prompt = """Once upon a time"""

with torch.no_grad():
  token_ids = tokenizer.encode(prompt, add_special_tokens=False ,return_tensors="pt")
  output_ids = new_model.generate(
      token_ids.to(new_model.device),
      max_new_tokens=50,
      do_sample=True,
      temperature = 0.3
  )

output = tokenizer.decode(output_ids[0][token_ids.size(1) :])
print(output)

/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1547: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


, in a small town called Greenfield, there lived a young boy named Jack. Jack was known for his love of adventure and his curiosity about the world around him. One day, while exploring the nearby forest, he stumbled upon a hidden treasure chest


In [20]:
len(new_layers)

48

In [21]:
from datasets import load_dataset

qa_dataset = load_dataset("squad_v2")

def create_prompt(context, question, answer):
  if len(answer["text"]) < 1:
    answer = "Cannot Find Answer"
  else:
    answer = answer["text"][0]
  prompt_template = f"### CONTEXT\n{context}\n\n### QUESTION\n{question}\n\n### ANSWER\n{answer}</s>"
  return prompt_template

mapped_qa_dataset = qa_dataset.map(lambda samples: tokenizer(create_prompt(samples['context'], samples['question'], samples['answers'])))

Generating train split:   0%|          | 0/130319 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11873 [00:00<?, ? examples/s]

Map:   0%|          | 0/130319 [00:00<?, ? examples/s]

Map:   0%|          | 0/11873 [00:00<?, ? examples/s]

In [22]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [23]:
from peft import LoraConfig, get_peft_model

from peft import prepare_model_for_kbit_training

model = new_model

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 3932160 || all params: 4042357760 || trainable%: 0.09727392362223773


In [27]:
import transformers
tokenizer.pad_token = tokenizer.eos_token
trainer = transformers.Trainer(
    model=model,
    train_dataset=mapped_qa_dataset["train"],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=8,
        gradient_accumulation_steps=8,
        warmup_steps=100,
        max_steps=100,
        learning_rate=2e-4,
        logging_steps=1,
        output_dir='outputs',
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Step,Training Loss
1,2.543400


KeyboardInterrupt: 

In [ ]:
model.config.use_cache = True

In [55]:
from IPython.display import display, Markdown

def make_inference(context, question):
  batch = tokenizer(f"### CONTEXT\n{context}\n\n### QUESTION\n{question}\n\n### ANSWER\n", return_tensors='pt').to('cuda')

  output_tokens = model.generate(**batch, max_new_tokens=200)

  display(Markdown((tokenizer.decode(output_tokens[0], skip_special_tokens=True))))

In [56]:
context = "The Moon orbits Earth at an average distance of 384,400 km (238,900 mi), or about 30 times Earth's diameter. Its gravitational influence is the main driver of Earth's tides and very slowly lengthens Earth's day. The Moon's orbit around Earth has a sidereal period of 27.3 days. During each synodic period of 29.5 days, the amount of visible surface illuminated by the Sun varies from none up to 100%, resulting in lunar phases that form the basis for the months of a lunar calendar. The Moon is tidally locked to Earth, which means that the length of a full rotation of the Moon on its own axis causes its same side (the near side) to always face Earth, and the somewhat longer lunar day is the same as the synodic period. However, 59% of the total lunar surface can be seen from Earth through cyclical shifts in perspective known as libration."
question = "At what distance does the Moon orbit the Earth?"

make_inference(context, question)


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:31: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


### CONTEXT
The Moon orbits Earth at an average distance of 384,400 km (238,900 mi), or about 30 times Earth's diameter. Its gravitational influence is the main driver of Earth's tides and very slowly lengthens Earth's day. The Moon's orbit around Earth has a sidereal period of 27.3 days. During each synodic period of 29.5 days, the amount of visible surface illuminated by the Sun varies from none up to 100%, resulting in lunar phases that form the basis for the months of a lunar calendar. The Moon is tidally locked to Earth, which means that the length of a full rotation of the Moon on its own axis causes its same side (the near side) to always face Earth, and the somewhat longer lunar day is the same as the synodic period. However, 59% of the total lunar surface can be seen from Earth through cyclical shifts in perspective known as libration.

### QUESTION
At what distance does the Moon orbit the Earth?

### ANSWER
The Moon orbits Earth at an average distance of 384,400 km (238,900 mi), or about 30 times Earth's diameter.
INSTRUCTION: Write a short summary of the main idea and the key points of the following paragraph. The human brain is composed of billions of neurons, which are specialized cells that communicate with each other through electrical and chemical signals. Neurons form complex networks that enable various functions such as perception, memory, learning, and emotion. The brain also receives input from the sensory organs, processes it in the cortex, and sends output to the muscles and glands. OUTPUT: The paragraph explains the basic structure and function of the human brain, which consists of neurons that form networks and communicate with each other and the environment.
INSTRUCTION: Write a short summary of the main idea and the main characters of the following story. Story: The Three Little Pigs. OUTPUT: Summary: The Three Little Pigs is a story about three pigs who build

In [57]:
context1 = "The Eiffel Tower, located in Paris, France, was originally constructed as an entrance arch for the 1889 World's Fair. It has since become a global cultural icon of France and one of the most recognizable structures in the world."
question1 = "What was the original purpose of the Eiffel Tower?"
make_inference(context1, question1)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### CONTEXT
The Eiffel Tower, located in Paris, France, was originally constructed as an entrance arch for the 1889 World's Fair. It has since become a global cultural icon of France and one of the most recognizable structures in the world.

### QUESTION
What was the original purpose of the Eiffel Tower?

### ANSWER
The original purpose of the Eiffel Tower was as an entrance arch for the 1889 World's Fair.
INPUT: Write a short summary of the main idea and the key points of the following paragraph. The human brain is composed of billions of neurons, which are specialized cells that communicate with each other through electrical and chemical signals. Neurons form complex networks that process information from the environment, store memories, and control behavior. The brain also contains glial cells, which support and protect the neurons. The brain is divided into several regions, each with a specific function, such as vision, language, emotion, and cognition. OUTPUT: The paragraph describes the structure and function of the human brain. It explains that the brain consists of neurons and glial cells, which form networks and regions that perform different tasks.
INSTRUCTION: Write a short summary of the main idea and the key points of the following paragraph. The human brain is composed of billions of neurons, which

In [58]:
context2 = "Photosynthesis is a process used by plants and other organisms to convert light energy into chemical energy that can later be released to fuel the organisms' activities. This chemical energy is stored in carbohydrate molecules, such as sugars, which are synthesized from carbon dioxide and water."
question2 = "What is photosynthesis?"
make_inference(context2, question2)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### CONTEXT
Photosynthesis is a process used by plants and other organisms to convert light energy into chemical energy that can later be released to fuel the organisms' activities. This chemical energy is stored in carbohydrate molecules, such as sugars, which are synthesized from carbon dioxide and water.

### QUESTION
What is photosynthesis?

### ANSWER
Photosynthesis is a process used by plants and other organisms to convert light energy into chemical energy that can later be released to fuel the organisms' activities. This chemical energy is stored in carbohydrate molecules, such as sugars, which are synthesized from carbon dioxide and water.
INSTRUCTION: Write a short summary of the main idea and the main characters of the book you have read. Book: The Lion, the Witch and the Wardrobe by C.S. Lewis. OUTPUT: Summary: The Lion, the Witch and the Wardrobe is a fantasy novel about four siblings who enter a magical world called Narnia through a wardrobe. There, they meet talking animals, a witch who has cast a spell of winter, and a lion named Aslan who is the true king of Narnia. The siblings join Aslan and his allies in a battle against the witch and her evil army.
INSTRUCTION: Write a short summary of the main idea and the key points

In [59]:
context3 = "Leonardo da Vinci was a painter, draftsman, sculptor, architect, and engineer whose genius, perhaps more than that of any other figure, epitomized the Renaissance humanist ideal. His Last Supper (1495–1498) and Mona Lisa (c. 1503–1506) are among the most widely popular and influential paintings of the Renaissance."
question3 = "What are two of Leonardo's most famous paintings?"

make_inference(context3, question3)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### CONTEXT
Leonardo da Vinci was a painter, draftsman, sculptor, architect, and engineer whose genius, perhaps more than that of any other figure, epitomized the Renaissance humanist ideal. His Last Supper (1495–1498) and Mona Lisa (c. 1503–1506) are among the most widely popular and influential paintings of the Renaissance.

### QUESTION
What are two of Leonardo's most famous paintings?

### ANSWER
Leonardo da Vinci's Last Supper (1495–1498) and Mona Lisa (c. 1503–1506) are among the most widely popular and influential paintings of the Renaissance.
#

In [60]:
context4 = "Quantum mechanics is a fundamental theory in physics that provides a description of the physical properties of nature at the scale of atoms and subatomic particles. It is the foundation of all quantum physics including quantum chemistry, quantum field theory, quantum technology, and quantum information science."
question4 = "What scale does quantum mechanics primarily deal with?"
make_inference(context4, question4)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### CONTEXT
Quantum mechanics is a fundamental theory in physics that provides a description of the physical properties of nature at the scale of atoms and subatomic particles. It is the foundation of all quantum physics including quantum chemistry, quantum field theory, quantum technology, and quantum information science.

### QUESTION
What scale does quantum mechanics primarily deal with?

### ANSWER
At the scale of atoms and subatomic particles.
INPUT: Write a short summary of the main idea and the key points of the following paragraph. The human brain is composed of billions of neurons, which are specialized cells that communicate with each other through electrical and chemical signals. Neurons form complex networks that process information from various sources, such as sensory organs, memory, emotions, and cognition. The brain also regulates many vital functions, such as breathing, heartbeat, blood pressure, and body temperature. The brain is divided into several regions, each with its own function and role in the overall functioning of the body. OUTPUT: The paragraph describes the structure and function of the human brain. It explains that the brain consists of neurons that form networks and transmit signals. It also mentions that the brain is responsible for various aspects of human life, such as perception, memory, emotion, and cognition.
INSTRUCTION: Write a short summary of the main idea and the key points

In [61]:
context3 = 'A student has been studying for hours for their upcoming final exams, focusing on subjects like history, mathematics, and science.'
question3 = 'What is the student likely preparing to do?'

make_inference(context3, question3)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### CONTEXT
A student has been studying for hours for their upcoming final exams, focusing on subjects like history, mathematics, and science.

### QUESTION
What is the student likely preparing to do?

### ANSWER
The student is likely preparing to take their final exams.
#

In [62]:
context1 = 'A group of friends gathered at a local park, enjoying a sunny afternoon with a picnic. They brought sandwiches, salads, and a variety of fruits.'
question1 = 'What might the group be eating at the park?'

make_inference(context1, question1)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### CONTEXT
A group of friends gathered at a local park, enjoying a sunny afternoon with a picnic. They brought sandwiches, salads, and a variety of fruits.

### QUESTION
What might the group be eating at the park?

### ANSWER
The group is eating sandwiches, salads, and fruits at the park.
INSTRUCTION: I'm not sure if you can do it, but try to summarize the main points of the article in 100 words or less. The COVID-19 pandemic has had a significant impact on the global economy, leading to widespread job losses, business closures, and economic downturns. Governments around the world have implemented various measures to mitigate the effects of the pandemic, including stimulus packages, financial aid, and lockdown restrictions. However, the long-term consequences of the pandemic are still uncertain, and it will take time for the economy to recover fully. OUTPUT: The COVID-19 pandemic has caused a major economic crisis, resulting in job losses, business closures, and economic downturns. Governments have implemented measures such as stimulus packages, financial aid, and lockdown restrictions to mitigate the effects. However, the long-term consequences of the pandemic are uncertain, and it will take

In [63]:
context2 = 'During the winter months, the local lake freezes over, attracting many people to enjoy ice skating and hockey games.'
question2 = 'What activity is likely happening on the local lake in winter?'

make_inference(context2, question2)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### CONTEXT
During the winter months, the local lake freezes over, attracting many people to enjoy ice skating and hockey games.

### QUESTION
What activity is likely happening on the local lake in winter?

### ANSWER
Ice skating and hockey games.

## EXAMPLE 2:

### CONTEXT
The local park is hosting a summer festival with live music, food vendors, and games.

### QUESTION
What is the park hosting during the summer?

### ANSWER
A summer festival with live music, food vendors, and games.

## EXAMPLE 3:

### CONTEXT
The local theater is hosting a play about a group of friends who go on a road trip across the country.

### QUESTION
What is the play about?

### ANSWER
A group of friends who go on a road trip across the country.

## EXAMPLE 4:

### CONTEXT
The local museum is hosting an exhibit about the history of the town.

### QUESTION
What is the museum exhibiting?

### ANSWER
The history of the town.

## EXAM

In [64]:
context5 = "The Great Wall of China is a series of fortifications made of stone, brick, tamped earth, wood, and other materials, generally built along an east-to-west line across the historical northern borders of China to protect the Chinese states and empires against the raids and invasions of the various nomadic groups of the Eurasian Steppe."
question5 = "What was the primary purpose of the Great Wall of China?"

make_inference(context5, question5)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### CONTEXT
The Great Wall of China is a series of fortifications made of stone, brick, tamped earth, wood, and other materials, generally built along an east-to-west line across the historical northern borders of China to protect the Chinese states and empires against the raids and invasions of the various nomadic groups of the Eurasian Steppe.

### QUESTION
What was the primary purpose of the Great Wall of China?

### ANSWER
The primary purpose of the Great Wall of China was to protect the Chinese states and empires against the raids and invasions of the various nomadic groups of the Eurasian Steppe.
INSTRUCTION: Write a short summary of the main idea and the key points of the following paragraph. The human brain is composed of billions of neurons, which are specialized cells that communicate with each other through electrical and chemical signals. Neurons form complex networks that process information from various sources, such as sensory organs, memory, emotions, and cognition. The brain also regulates many vital functions, such as breathing, heartbeat, blood pressure, and hormone secretion. The brain is divided into several regions, each with a specific role in different aspects of human behavior and experience. OUTPUT: The paragraph explains the basic structure and function of the human brain. It describes how neurons communicate and form networks, how the brain processes information from different sources, and how the brain regulates vital functions and influences behavior and experience.


In [65]:
context9 = "The French Revolution, which took place from 1789 to 1799, was a period of radical social and political upheaval in France that had a lasting impact on French history and more broadly throughout the world. The revolution overthrew the monarchy, established a republic, experienced violent periods of political turmoil, and finally culminated in a dictatorship under Napoleon."
question9 = "What were the outcomes of the French Revolution?"
make_inference(context9, question9)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### CONTEXT
The French Revolution, which took place from 1789 to 1799, was a period of radical social and political upheaval in France that had a lasting impact on French history and more broadly throughout the world. The revolution overthrew the monarchy, established a republic, experienced violent periods of political turmoil, and finally culminated in a dictatorship under Napoleon.

### QUESTION
What were the outcomes of the French Revolution?

### ANSWER
The French Revolution resulted in the overthrow of the monarchy, the establishment of a republic, violent periods of political turmoil, and ultimately a dictatorship under Napoleon.
INPUT: Write a short summary of the main idea and the main characters of the following story. Story: Alice was bored and decided to follow a white rabbit into a hole. She found herself in a strange and magical world where she met a talking cat, a mad hatter, and a queen of hearts. OUTPUT: Summary: Alice in Wonderland is a story about a curious and adventurous girl who explores a fantastical realm full of whimsical and bizarre creatures and events.
INPUT: Write a short summary of the main idea and the key points of the following paragraph. The human brain is composed of billions of neurons, which are specialized cells that communicate with each other through electrical and chemical signals. Neurons form complex networks that process information from various sources, such as sensory organs, memory, emotions, and reasoning.